# Базовый курс по методам анализа данных и машинного обучения

## Практическое занятие 2: Сплайн интерполяция. Аппрокисмация
<br><br><br><br>
__Аксентьев Артем (akseart@ya.ru)__

__Ксемидов Борис (nstalker.anonim@yandex.ru)__
<br>

In [228]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Сплайн интерполяция

На каждом отрезке $[x_{i - 1},x_{i}],\ i=\overline{1,N}$ функция $S(x)$ есть полином третьей степени $S_i(x)$, коэффициенты которого надо определить. Запишем для удобства $S_i(x)$ в виде:

$$S_i(x) = a_i + b_i(x - x_i) + {c_i}(x-x_i)^2 + {d_i}(x - x_i)^3$$

тогда

$$S_i\left(x_i\right) = a_i, \quad S'_i(x_i) = b_i, \quad S''_i(x_i) = 2c_i, \quad
S'''_i\left(x_i\right) = 6d_i \quad i=\overline{1,N}.$$

Условия непрерывности всех производных до второго порядка включительно
записываются в виде <br />
$$S_i\left(x_{i-1}\right) = S_{i-1}(x_{i-1}),$$
$$S'_i\left(x_{i-1}\right) = S'_{i-1}(x_{i-1}),$$
$$S''_i\left(x_{i-1}\right) = S''_{i-1}(x_{i-1}),$$

где $i$ меняется от $1$ до $N,$ а условия интерполяции в виде
$$S_i\left(x_{i}\right) = f(x_{i}).$$

Обозначим: $\quad h_i = x_i - x_{i-1}\quad (i = \overline{1,N}), \quad f_{i} = f(x_{i})\quad (i = \overline{0,N})$

Отсюда получаем формулы для вычисления коэффициентов "Естественного сплайна":
$$a_{i} = f(x_{i});$$
$$d_{i} = \frac{c_{i} - c_{i - 1}}{3 \cdot h_{i}};$$
$$b_{i} = \frac{a_{i} - a_{i - 1}}{h_{i}} + \frac{2 \cdot c_{i} + c_{i - 1}}{3} \cdot h_{i};$$
$$c_{i - 1} \cdot h_{i} + 2 \cdot c_{i} \cdot(h_{i} + h_{i+1}) + c_{i + 1} \cdot h_{i+1} = 3 \cdot \left(\frac{a_{i+1} - a_{i}}{h_{i+1}} - \frac{a_{i} - a_{i - 1}}{h_{i}}\right),$$
причем $c_{N} = S''(x_{N}) = 0$ и $c_{1} - 3 \cdot d_{1} \cdot h_{1} = S''(x_{0}) = 0$.



In [229]:
class Spline:
    def __init__(self, x, y):
        self.X = x
        self.Y = y
        self.h = self.X[1] - self.X[0]

        coef = np.zeros((len(self.X), len(self.X)))
        f = np.zeros(len(self.X))
        coef[0, 0] = 1
        coef[-1, -1] = 1

        for i in range(1, len(self.X) - 1):
            coef[i, i - 1] = self.h / 6
            coef[i, i] = 2 * self.h / 3
            coef[i, i + 1] = self.h / 6

            f[i] = (self.Y[i+1] - self.Y[i]) / self.h - (self.Y[i] - self.Y[i - 1]) / self.h

        self.c = np.linalg.solve(coef, f)

    def __call__(self, x):
        num_polynom = len(self.X)
        for i, X in enumerate(self.X):
            if x <= X:
                num_polynom = i
                break

        result = (self.c[num_polynom - 1] * (self.X[num_polynom] - x) ** 3 / (6 * self.h) +
                  self.c[num_polynom] * (x - self.X[num_polynom - 1]) ** 3 / (6 * self.h) +
                  (self.Y[num_polynom - 1] / self.h - self.c[num_polynom - 1] * self.h / 6) * (self.X[num_polynom] - x) +
                  (self.Y[num_polynom] / self.h - self.c[num_polynom] * self.h / 6) * (x - self.X[num_polynom - 1]))
        return result
        

In [230]:
x = np.linspace(0, 4, 10)
def foo(x):
    return np.sin(3*x)

y = foo(x)


In [231]:
sp = Spline(x, y)

In [233]:
x, y

(array([0.        , 0.44444444, 0.88888889, 1.33333333, 1.77777778,
        2.22222222, 2.66666667, 3.11111111, 3.55555556, 4.        ]),
 array([ 0.        ,  0.9719379 ,  0.45727263, -0.7568025 , -0.81332939,
         0.37415123,  0.98935825,  0.09131724, -0.94639576, -0.53657292]))

In [235]:
sp(4)

-0.5365729180004352

In [240]:
fig = go.Figure()
x1 = np.linspace(0, 4, 400)
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name="Interpolations point"))
fig.add_trace(go.Scatter(x=x1, y=foo(x1), name="original function"))
fig.add_trace(go.Scatter(x=x1, y =[sp(x) for x in x1], name="Interpolations line"))

fig.show()

# МНК

Пусть $x$ — набор $n$ неизвестных переменных (параметров).

$f_i(x)$, $i=1, \ldots, m$, $m>n$— совокупность функций от этого набора переменных.

Задача заключается в подборе таких значений $x$, чтобы значения этих функций были максимально близки к некоторым значениям $y_i$. По существу речь идет о «решении» переопределённой системы уравнений $f_i(x)=y_i$, $i=1, \ldots, m$ в указанном смысле максимальной близости левой и правой частей системы. Суть МНК заключается в выборе в качестве «меры близости» суммы квадратов отклонений левых и правых частей $|f_i(x)-y_i|$. Таким образом, сущность МНК может быть выражена следующим образом:

$$\sum_i e^2_i=\sum_i (y_i-f_i(x))^2 \rightarrow \min_x$$

данные аппроксимируются полиномиальной функцией регрессии одной переменной

$$f(x)=b_{0}+\sum \limits _{i=1}^{k}b_{i}x^{i}$$

$\left(\begin{array}{cccc}n & \sum_{n} x_{t} & \cdots & \sum_{n} x_{t}^{k} \\ \sum_{n} x_{t} & \sum_{n} x_{t}^{2} & \ldots & \sum_{n} x_{t}^{k+1} \\ \vdots & \vdots & \ddots & \vdots \\ \sum_{n} x_{t}^{k} & \sum_{n} x_{t}^{k+1} & \ldots & \sum_{n} x_{t}^{2 k}\end{array}\right)\left[\begin{array}{c}b_{0} \\ b_{1} \\ \vdots \\ b_{k}\end{array}\right]=\left[\begin{array}{c}\sum_{n} y_{t} \\ \sum_{n} x_{t} y_{t} \\ \vdots \\ \sum_{n} x_{t}^{k} y_{t}\end{array}\right] .$


$Ax=b$, где $A$ прямоугольная матрица размера $m\times n,m>n$ (то есть число строк матрицы A больше количества искомых переменных).

Такая система уравнений в общем случае не имеет решения. Поэтому эту систему можно «решить» только в смысле выбора такого вектора $x$, чтобы минимизировать «расстояние» между векторами $Ax$ и $b$. Для этого можно применить критерий минимизации суммы квадратов разностей левой и правой частей уравнений системы, то есть $ (Ax-b)^{T}(Ax-b)\rightarrow \min _{x}$. Нетрудно показать, что решение этой задачи минимизации приводит к решению следующей системы уравнений

$$A^{T}Ax=A^{T}b\Rightarrow x=(A^{T}A)^{{-1}}A^{T}b.$$

In [241]:
class MLS:
    def __init__(self, X, Y, m=1):
        self._pow = np.arange(0, m + 1, 1)
        self._compute_mult(X.reshape((X.shape[0], 1)), Y.reshape((X.shape[0], 1)))

    def __call__(self, x):
        return np.dot(self._mult, (x ** self._pow)[:, np.newaxis])

    def _compute_mult(self, X, Y):
        X = X ** self._pow
        self._mult = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(Y).flatten()

In [242]:
nprand = np.random.RandomState(42)

def func(x):
    return np.sin(3*x) + nprand.normal(scale=0.1, size=x.shape)

X = np.linspace(1, 4, 15)
Y_func = func(X)

In [243]:
X, Y_func

(array([1.        , 1.21428571, 1.42857143, 1.64285714, 1.85714286,
        2.07142857, 2.28571429, 2.5       , 2.71428571, 2.92857143,
        3.14285714, 3.35714286, 3.57142857, 3.78571429, 4.        ]),
 array([ 0.19079142, -0.49436128, -0.84557809, -0.82442045, -0.67658034,
        -0.09225879,  0.70088096,  1.01474345,  0.91161738,  0.65070016,
        -0.05013523, -0.6490896 , -0.93650231, -1.12667113, -0.7090647 ]))

In [244]:
np.sin(3*X)

array([ 0.14112001, -0.48053485, -0.91034694, -0.97672344, -0.65316501,
       -0.06884509,  0.54295968,  0.93799998,  0.95856482,  0.59644416,
       -0.00379346, -0.60251662, -0.96069854, -0.93534311, -0.53657292])

In [258]:
splain_pow = range(1, 5)

In [259]:
x1 = np.linspace(1, 4, 400)
y1 = np.sin(3*x1)
fig = make_subplots(rows=len(splain_pow),
                    cols=2,
                    subplot_titles=[f"Аппроксимация {x}" if not i else
                                    f"Погрешность {x}"
                                      for x in splain_pow for i in range(2)]
                    )
for i, s_pow in enumerate(splain_pow):
    result = MLS(X, Y_func, s_pow)
    y_res = [float(result(i)) for i in x1]
    fig.add_trace(go.Scatter(x=X, y=Y_func, mode='markers', name="Point"), i+1, 1)
    fig.add_trace(go.Scatter(x=x1, y=y1, name="original function"), i+1, 1)
    fig.add_trace(go.Scatter(x=x1, y=y_res, name=f"Approx line {s_pow}"),i+1, 1)
    fig.add_trace(go.Scatter(x=x1, y=y1-y_res, name=f"Error{s_pow}"), i+1, 2)

fig.update_layout(hovermode="x",
                  margin=dict(l=0, r=0, t=50, b=50),
                  height=len(splain_pow) * 250,
                  width=1000)
fig.show()

In [260]:
def rmse(x, y):
    return np.sqrt(np.mean((x - y) ** 2))

rmse(y1, y_res)

0.16678287591443344

# Задание(срок сдачи воскресенье 23:59)):
Для каждой функции:
   - $\sin(x)$
   - $\cos(10x)$
   - $x^2$
   - $e^x$ np.e

Взять 10 точек и по ним, с помощью модуля scipy и numpy реализовать:
   - Сплайн интерполяция ([https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html#scipy.interpolate.CubicSpline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html#scipy.interpolate.CubicSpline))
   - Интерполяцию Лагранжа ([https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.lagrange.html?highlight=lagrange#scipy.interpolate.lagrange](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.lagrange.html?highlight=lagrange#scipy.interpolate.lagrange))
   - МНК(добавив какой-либо <<шум>>) ([https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html))

Для каждой функции должен быть отрисован график с точками, исходная функция и полученная функция. Если поведение графика очень сильно отличается от условно-корректного, то необходимо описать почему это происходит и предложить методы решения этой проблемы

# Рекомендуемая литература:
- http://mathprofi.ru/metod_naimenshih_kvadratov.html
- https://towardsdatascience.com/least-squares-linear-regression-in-python-54b87fc49e77
